# Segmentación semántica con Segmentnet

La siguiente capacidad de aprendizaje profundo que cubriremos en estos apuntes es la segmentación semántica. La segmentación semántica se basa en el reconocimiento de imágenes, excepto que las clasificaciones ocurren a nivel de píxel en oposición a la imagen completa. Esto se logra convolucionalizando una backbone de reconocimiento de imágenes previamente entrenada, que transforma el modelo en una Red totalmente convolucional (FCN) capaz de etiquetado por píxel. Especialmente útil para la percepción ambiental, la segmentación produce clasificaciones densas por píxel de muchos objetos potenciales diferentes por escena, incluidos los primeros planos y fondos de las escenas.

`segNet` acepta como entrada la imagen 2D y genera una segunda imagen con la superposición de máscara de clasificación por píxel. Cada píxel de la máscara corresponde a la clase de objeto que se clasificó.

La lista de objetos que se pueden detectar mediante las redes preentrenadas se encuentra en este [enlace](https://github.com/dusty-nv/jetson-inference/blob/master/data/networks/ssd_coco_labels.txt)

--------------

## Uso de los programas precompilados de la Jetson

Aquí se pueden encontrar los códigos de los programas precompilados:
 * [C++](https://github.com/dusty-nv/jetson-inference/blob/master/examples/segnet/segnet.cpp)
 * [Python](https://github.com/dusty-nv/jetson-inference/blob/master/python/examples/segnet.py)

--------------

## Modelos de segmentación pre-entrenados disponibles

A continuación se verán varios modelos de segmentación previamente entrenados disponibles que utilizan la red FCN-ResNet18 con rendimiento en tiempo real en Jetson. Se proporcionan modelos para una variedad de entornos y temas, incluidas ciudades urbanas, senderos todo terreno y espacios de oficinas y hogares en interiores.

A continuación se muestra una tabla de los modelos de segmentación semántica previamente entrenados disponibles y el argumento --network asociado que se segnetutilizará para cargarlos. Están basados ​​en la red FCN-ResNet18 de clase 21 y han sido entrenados en varios conjuntos de datos y resoluciones usando [PyTorch](https://github.com/dusty-nv/pytorch-segmentation), y fueron exportados al [formato ONNX](https://onnx.ai/) para cargarlos con TensorRT.

|Dataset|Resolution|Resolution|Accuracy (%)|Jetson Nano (FPS)|Jetson Xavier (FPS)|
|-------|----------|----------|--------|-----------|-------------|
|[Cityscapes](https://www.cityscapes-dataset.com/)|512x256|``fcn-resnet18-cityscapes-512x256``|83.3|48|480|
|[Cityscapes](https://www.cityscapes-dataset.com/)|1024x512|``fcn-resnet18-cityscapes-1024x512``|87.3|12|175|
|[Cityscapes](https://www.cityscapes-dataset.com/)|2048x1024|``fcn-resnet18-cityscapes-2048x1024``|89.6|3|47|
|[DeepScene](http://deepscene.cs.uni-freiburg.de/)|576x320|``fcn-resnet18-deepscene-576x320``|96.4|26|360|
|[DeepScene](http://deepscene.cs.uni-freiburg.de/)|864x480|``fcn-resnet18-deepscene-864x480``|96.9|14|190|
|[Multi-Human](https://lv-mhp.github.io/)|512x320|``fcn-resnet18-mhp-512x320``|86.5|34|370|
|[Multi-Human](https://lv-mhp.github.io/)|640x360|``fcn-resnet18-mhp-640x360``|87.1|23|325|
|[Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/)|320x320|``fcn-resnet18-voc-320x320``|85.9|45|508|
|[Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/)|512x320|``fcn-resnet18-voc-512x320``|88.5|34|375|
|[SUN RGB-D](http://rgbd.cs.princeton.edu/)|512x400|``fcn-resnet18-sun-512x400``|64.3|28|340|
|[SUN RGB-D](http://rgbd.cs.princeton.edu/)|640x512|``fcn-resnet18-sun-640x512``|65.1|17|224|

 * Si se omite la resolución del argumento CLI, se carga el modelo de resolución más baja
 * La precisión indica la precisión de la clasificación de píxeles en el conjunto de datos de validación del modelo.
 * El rendimiento se mide para el modo GPU FP16 con JetPack 4.2.1, `nvpmodel 0` (MAX-N)

 > **nota**: para descargar redes adicionales, ejecute la herramienta Model Downloader
 
 >           ``$ cd jetson-inference/tools``
 >           ``$ ./download-models.sh``

## Uso de los programas precompilados de la Jetson

Además de las rutas de entrada/salida, hay algunas opciones de línea de comandos adicionales:

 * flag ``--network`` (opcional) cambia el modelo de segmentación que se está utilizando (ver arriba)
 * flag ``--visualize`` (opcional) acepta ``mask`` y/o ``overlay`` (el valor predeterminado es ``overlay,mask``)
 * flag ``--alpha`` (opcional) establece el valor de fusión alfa para ``overlay`` (el valor predeterminado es 120)
 * flag ``--filter-mode`` (opcional) acepta ``point`` o ``linear`` (el valor predeterminado es ``linear``)

Usar el flag ``--help`` para obtener más información

Ir a la carpeta con los programas precompilados con el siguiente comando

```
$ cd jetson-inference/build/aarch64/bin
```

Comandos:

```
# C++
$ $ ./segnet --network=<model> input.jpg output.jpg

# Python
$ ./segnet.py --network=<model> input.jpg output.jpg
```

### Cityscapes

A continuación, se muestra un ejemplo de segmentación de una escena de una calle urbana con el modelo [Cityscapes](https://www.cityscapes-dataset.com/).

```
# C++
$ ./segnet --network=fcn-resnet18-cityscapes images/city_0.jpg images/test/city_0.jpg

# Python
$ ./segnet.py --network=fcn-resnet18-cityscapes images/city_0.jpg images/test/city_0.jpg
```

![Cityscapes](https://raw.githubusercontent.com/dusty-nv/jetson-inference/pytorch/docs/images/segmentation-city.jpg)

### DeepScene

El conjunto de datos [DeepScene](http://deepscene.cs.uni-freiburg.de/) consta de senderos forestales y vegetación fuera de la carretera, lo que ayuda a los robots al aire libre en el seguimiento de caminos.

```
# C++
$ ./segnet --network=fcn-resnet18-deepscene images/trail_0.jpg images/test/trail_0.jpg

# Python
$ ./segnet.py --network=fcn-resnet18-deepscene images/trail_0.jpg images/test/trail_0.jpg
```

![DeepScene](https://raw.githubusercontent.com/dusty-nv/jetson-inference/pytorch/docs/images/segmentation-deepscene-0-overlay.jpg)

### Multi-Human Parsing (MHP)

[Multi-Human Parsing](https://lv-mhp.github.io/) proporciona un etiquetado denso de las partes del cuerpo, como brazos, piernas, cabeza y diferentes tipos de ropa.

```
# C++
$ ./segnet --network=fcn-resnet18-mhp images/humans_0.jpg images/test/humans_0.jpg

# Python
$ ./segnet.py --network=fcn-resnet18-mhp images/humans_0.jpg images/test/humans_0.jpg
```

![segmentation-mhp-0](https://raw.githubusercontent.com/dusty-nv/jetson-inference/pytorch/docs/images/segmentation-mhp-0.jpg)

![segmentation-mhp-1](https://raw.githubusercontent.com/dusty-nv/jetson-inference/pytorch/docs/images/segmentation-mhp-1.jpg)

![segmentation-mhp-legend](https://raw.githubusercontent.com/dusty-nv/jetson-inference/pytorch/docs/images/segmentation-mhp-legend.jpg)

### Pascal VOC

[Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/) es uno de los conjuntos de datos originales utilizados para la segmentación semántica, que contiene varias personas, animales, vehículos y objetos domésticos.

```
# C++
$ ./segnet --network=fcn-resnet18-voc images/object_0.jpg images/test/object_0.jpg

# Python
$ ./segnet.py --network=fcn-resnet18-voc images/object_0.jpg images/test/object_0.jpg
```

![segmentation-voc](https://raw.githubusercontent.com/dusty-nv/jetson-inference/pytorch/docs/images/segmentation-voc.jpg)

![segmentation-voc-legend](https://raw.githubusercontent.com/dusty-nv/jetson-inference/pytorch/docs/images/segmentation-voc-legend.jpg)

### SUN RGB-D

El conjunto de datos [SUN RGB-D](http://rgbd.cs.princeton.edu/) proporciona segmentación del terreno para muchos objetos y escenas interiores que se encuentran comúnmente en espacios de oficinas y hogares.

```
# C++
$ ./segnet --network=fcn-resnet18-sun images/room_0.jpg images/test/room_0.jpg

# Python
$ ./segnet.py --network=fcn-resnet18-sun images/room_0.jpg images/test/room_0.jpg
```

![segmentation-sun](https://raw.githubusercontent.com/dusty-nv/jetson-inference/pytorch/docs/images/segmentation-sun.jpg)

![segmentation-sun-legend](https://raw.githubusercontent.com/dusty-nv/jetson-inference/pytorch/docs/images/segmentation-sun-legend.jpg)

### Procesamiento de varias imágenes

Si queremos detectar varias imágenes

```
# C++
$ ./segnet --network=fcn-resnet18-sun "images/room_*.jpg" "images/test/room_%i.jpg"

# Python
$ ./segnet.py --network=fcn-resnet18-sun "images/room_*.jpg" "images/test/room_%i.jpg"
```

 > **nota**: cuando se usen asteriscos, hay que escribirlos siempre entre comillas ("*.jpg"). De lo contrario, el sistema operativo expandirá automáticamente la secuencia y modificará el orden de los argumentos en la línea de comandos, lo que puede resultar en que una de las imágenes de entrada sea sobrescrita por la salida.

### Visualización

Si queremos cambiar eñ tipo de visualización hay que usar el flag `--visualize`. Podemos usar `mask` y `overlay` (por defecto `overlay,mask`)

```
# C++
$ ./segnet --network=fcn-resnet18-mhp --visualize=mask images/humans_0.jpg images/test/humans_0.jpg         # Only mask
$ ./segnet --network=fcn-resnet18-mhp --visualize=overlay images/humans_1.jpg images/test/humans_1.jpg      # Only overlay

# Python
$ ./segnet.py --network=fcn-resnet18-mhp --visualize=mask images/humans_0.jpg images/test/humans_0.jpg      # Only mask
$ ./segnet.py --network=fcn-resnet18-mhp --visualize=overlay images/humans_1.jpg images/test/humans_1.jpg   # Only overlay
```

Transparencia

Si el modo de visualización es `overlay`, se puede elegir la transparencia de la máscara mediante el flag `--alpha`, cuanto más pequeño es el valor, más transparente es la máscara

```
# C++
$ ./segnet --network=fcn-resnet18-mhp --visualize=overlay --alpha=50 images/humans_0.jpg images/test/humans_0.jpg      # Alpha 50
$ ./segnet --network=fcn-resnet18-mhp --visualize=overlay --alpha=200 images/humans_1.jpg images/test/humans_1.jpg     # Alpha 200

# Python
$ ./segnet.py --network=fcn-resnet18-mhp --visualize=overlay --alpha=50 images/humans_0.jpg images/test/humans_0.jpg      # Alpha 50
$ ./segnet.py --network=fcn-resnet18-mhp --visualize=overlay --alpha=200 images/humans_1.jpg images/test/humans_1.jpg     # Alpha 200
```

### Tipo de filtro

Se puede elegir el tipo de filtro mediante el flag `--filter-mode`. Podemos usar `linear` o `point` (por defecto `linear`)

```
# C++
$ ./segnet --network=fcn-resnet18-mhp --filter-mode=linear images/humans_0.jpg images/test/humans_0.jpg    # Filter mode linear
$ ./segnet --network=fcn-resnet18-mhp --filter-mode=point images/humans_1.jpg images/test/humans_1.jpg     # Filter mode point

# Python
$ ./segnet.py --network=fcn-resnet18-mhp --filter-mode=linear images/humans_0.jpg images/test/humans_0.jpg    # Filter mode linear
$ ./segnet.py --network=fcn-resnet18-mhp --filter-mode=point images/humans_1.jpg images/test/humans_1.jpg     # Filter mode point
```

### Segmentación de vídeos

Si se quiere procesar un videdo solo hay que indicarlo en la entrada

Para ello ejecutamos el docker montando la carpeta del SDK de VisionWorks

```
$ docker/run.sh --volume /usr/share/visionworks/sources/data:/videos
```

Y ya lo podemos procesar

```
# C++
$ ./segnet --network=fcn-resnet18-cityscapes /videos/cars.mp4 images/test/cars_segmentation.mp4

# Python
$ ./segnet.py --network=fcn-resnet18-cityscapes /videos/cars.mp4 images/test/cars_segmentation.mp4
```

--------------

## Crear un programa de clasificación en Python

Como vamos a crear un programa, lo primero que tenemos que hacer es crear una carpeta en el Host donde guardaremos el programa

```
$ cd ~/
$ mkdir my-segmentation-python
$ cd my-segmentation-python
$ touch my-segmentation.py
$ chmod +x my-segmentation.py
```

Nos descargamos unas imágenes de osos para probar

```
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/black_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/brown_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/polar_bear.jpg
```

A continuación lo que hay que hacer es lanzar el Docker con una carpeta del Host compartida, para que así cuando se cierre el Docker no se borre el programa, para ello lanzamos el Docker con el siguiente comando

```
$ docker/run.sh --volume ~/my-segmentation-python:/my-segmentation-python   # mounted inside the container to /my-segmentation-python
```

Una vez dentro del Docker ir a la carpeta con los siguientes comandos

```
$ cd ../
$ cd my-segmentation-python
```

Editar el archivo .py con el siguiente comando

```
$ nano my-segmentation.py
```

Para crear un programa como el precompilado escribimos el siguiente código

```Python
import jetson.inference
import jetson.utils

import argparse
import sys

# Add /jetson-inference/python/examples path for import segnet_utils
sys.path.append('/jetson-inference/python/examples')
from segnet_utils import *

# parse the command line
parser = argparse.ArgumentParser(description="Segment a live camera stream using an semantic segmentation DNN.", 
                                 formatter_class=argparse.RawTextHelpFormatter, epilog=jetson.inference.segNet.Usage() +
                                 jetson.utils.videoSource.Usage() + jetson.utils.videoOutput.Usage() + jetson.utils.logUsage())

parser.add_argument("input_URI", type=str, default="", nargs='?', help="URI of the input stream")
parser.add_argument("output_URI", type=str, default="", nargs='?', help="URI of the output stream")
parser.add_argument("--network", type=str, default="fcn-resnet18-voc", help="pre-trained model to load, see below for options")
parser.add_argument("--filter-mode", type=str, default="linear", choices=["point", "linear"], help="filtering mode used during visualization, options are:\n  'point' or 'linear' (default: 'linear')")
parser.add_argument("--visualize", type=str, default="overlay,mask", help="Visualization options (can be 'overlay' 'mask' 'overlay,mask'")
parser.add_argument("--ignore-class", type=str, default="void", help="optional name of class to ignore in the visualization results (default: 'void')")
parser.add_argument("--alpha", type=float, default=150.0, help="alpha blending value to use during overlay, between 0.0 and 255.0 (default: 150.0)")
parser.add_argument("--stats", action="store_true", help="compute statistics about segmentation mask class output")

is_headless = ["--headless"] if sys.argv[0].find('console.py') != -1 else [""]

try:
	opt = parser.parse_known_args()[0]
except:
	print("")
	parser.print_help()
	sys.exit(0)

# load the segmentation network
net = jetson.inference.segNet(opt.network, sys.argv)

# set the alpha blending value
net.SetOverlayAlpha(opt.alpha)

# create video output
output = jetson.utils.videoOutput(opt.output_URI, argv=sys.argv+is_headless)

# create buffer manager
buffers = segmentationBuffers(net, opt)

# create video source
input = jetson.utils.videoSource(opt.input_URI, argv=sys.argv)

# process frames until user exits
while True:
	# capture the next image
	img_input = input.Capture()

	# allocate buffers for this size image
	buffers.Alloc(img_input.shape, img_input.format)

	# process the segmentation network
	net.Process(img_input, ignore_class=opt.ignore_class)

	# generate the overlay
	if buffers.overlay:
		net.Overlay(buffers.overlay, filter_mode=opt.filter_mode)

	# generate the mask
	if buffers.mask:
		net.Mask(buffers.mask, filter_mode=opt.filter_mode)

	# composite the images
	if buffers.composite:
		jetson.utils.cudaOverlay(buffers.overlay, buffers.composite, 0, 0)
		jetson.utils.cudaOverlay(buffers.mask, buffers.composite, buffers.overlay.width, 0)

	# render the output image
	output.Render(buffers.output)

	# update the title bar
	output.SetStatus("{:s} | Network {:.0f} FPS".format(opt.network, net.GetNetworkFPS()))

	# print out performance info
	jetson.utils.cudaDeviceSynchronize()
	net.PrintProfilerTimes()

    # compute segmentation class stats
	if opt.stats:
		buffers.ComputeStats()
    
	# exit on input/output EOS
	if not input.IsStreaming() or not output.IsStreaming():
		break

```

Ejecutar el programa con el siguiente comando

```
$ python3 my-segmentation.py /dev/video0
```

En este caso abrirá la webcam, se pueden introducir las mismas variables que con el programa precompilado

--------------

## Crear un programa ded clasificación en C++

Como vamos a crear un programa, lo primero que tenemos que hacer es crear una carpeta en el Host donde guardaremos el programa

```
$ cd ~/
$ mkdir my-segmentation-cpp
$ cd my-segmentation-cpp
$ touch my-segmentation.cpp
$ chmod +x my-segmentation.cpp
$ touch CMakeLists.txt
$ chmod +x CMakeLists.txt
```

Nos descargamos unas imágenes de osos para probar

```
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/black_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/brown_bear.jpg
$ wget https://github.com/dusty-nv/jetson-inference/raw/master/data/images/polar_bear.jpg
```

A continuación lo que hay que hacer es lanzar el Docker con una carpeta del Host compartida, para que así cuando se cierre el Docker no se borre el programa, para ello lanzamos el Docker con el siguiente comando

```
$ docker/run.sh --volume ~/my-segmentation-cpp:/my-segmentation-cpp   # mounted inside the container to /my-segmentation-cpp
```

Una vez dentro del Docker ir a la carpeta con los siguientes comandos

```
$ cd ../
$ cd my-segmentation-cpp
```

Editar el archivo .py con el siguiente comando

```
$ nano my-segmentation.cpp
```

Para crear un programa como el precompilado escribimos el siguiente código

```C++
#include <jetson-utils/videoSource.h>
#include <jetson-utils/videoOutput.h>

#include <jetson-utils/cudaFont.h>

#include <jetson-utils/cudaOverlay.h>
#include <jetson-utils/cudaMappedMemory.h>

#include <jetson-inference/segNet.h>

#include <signal.h>


#ifdef HEADLESS
	#define IS_HEADLESS() "headless"             // run without display
	#define DEFAULT_VISUALIZATION "overlay"      // output overlay only
#else
	#define IS_HEADLESS() (const char*)NULL      // use display (if attached)
	#define DEFAULT_VISUALIZATION "overlay|mask" // output overlay + mask
#endif


bool signal_recieved = false;

void sig_handler(int signo)
{
	if( signo == SIGINT )
	{
		LogVerbose("received SIGINT\n");
		signal_recieved = true;
	}
}

int usage()
{
	printf("usage: segnet [--help] [--network NETWORK] ...\n");
	printf("              input_URI [output_URI]\n\n");
	printf("Segment and classify a video/image stream using a semantic segmentation DNN.\n");
	printf("See below for additional arguments that may not be shown above.\n\n");
	printf("positional arguments:\n");
	printf("    input_URI       resource URI of input stream  (see videoSource below)\n");
	printf("    output_URI      resource URI of output stream (see videoOutput below)\n\n");

	printf("%s\n", segNet::Usage());
	printf("%s\n", videoSource::Usage());
	printf("%s\n", videoOutput::Usage());
	printf("%s\n", Log::Usage());

	return 0;
}


//
// segmentation buffers
//
typedef uchar3 pixelType;		// this can be uchar3, uchar4, float3, float4

pixelType* imgMask      = NULL;	// color of each segmentation class
pixelType* imgOverlay   = NULL;	// input + alpha-blended mask
pixelType* imgComposite = NULL;	// overlay with mask next to it
pixelType* imgOutput    = NULL;	// reference to one of the above three

int2 maskSize;
int2 overlaySize;
int2 compositeSize;
int2 outputSize;

// allocate mask/overlay output buffers
bool allocBuffers( int width, int height, uint32_t flags )
{
	// check if the buffers were already allocated for this size
	if( imgOverlay != NULL && width == overlaySize.x && height == overlaySize.y )
		return true;

	// free previous buffers if they exit
	CUDA_FREE_HOST(imgMask);
	CUDA_FREE_HOST(imgOverlay);
	CUDA_FREE_HOST(imgComposite);

	// allocate overlay image
	overlaySize = make_int2(width, height);
	
	if( flags & segNet::VISUALIZE_OVERLAY )
	{
		if( !cudaAllocMapped(&imgOverlay, overlaySize) )
		{
			LogError("segnet:  failed to allocate CUDA memory for overlay image (%ux%u)\n", width, height);
			return false;
		}

		imgOutput = imgOverlay;
		outputSize = overlaySize;
	}

	// allocate mask image (half the size, unless it's the only output)
	if( flags & segNet::VISUALIZE_MASK )
	{
		maskSize = (flags & segNet::VISUALIZE_OVERLAY) ? make_int2(width/2, height/2) : overlaySize;

		if( !cudaAllocMapped(&imgMask, maskSize) )
		{
			LogError("segnet:  failed to allocate CUDA memory for mask image\n");
			return false;
		}

		imgOutput = imgMask;
		outputSize = maskSize;
	}

	// allocate composite image if both overlay and mask are used
	if( (flags & segNet::VISUALIZE_OVERLAY) && (flags & segNet::VISUALIZE_MASK) )
	{
		compositeSize = make_int2(overlaySize.x + maskSize.x, overlaySize.y);

		if( !cudaAllocMapped(&imgComposite, compositeSize) )
		{
			LogError("segnet:  failed to allocate CUDA memory for composite image\n");
			return false;
		}

		imgOutput = imgComposite;
		outputSize = compositeSize;
	}

	return true;
}


int main( int argc, char** argv )
{
	/*
	 * parse command line
	 */
	commandLine cmdLine(argc, argv, IS_HEADLESS());

	if( cmdLine.GetFlag("help") )
		return usage();


	/*
	 * attach signal handler
	 */
	if( signal(SIGINT, sig_handler) == SIG_ERR )
		LogError("can't catch SIGINT\n");


	/*
	 * create input stream
	 */
	videoSource* input = videoSource::Create(cmdLine, ARG_POSITION(0));

	if( !input )
	{
		LogError("segnet:  failed to create input stream\n");
		return 0;
	}


	/*
	 * create output stream
	 */
	videoOutput* output = videoOutput::Create(cmdLine, ARG_POSITION(1));
	
	if( !output )
		LogError("segnet:  failed to create output stream\n");	
	

	/*
	 * create segmentation network
	 */
	segNet* net = segNet::Create(cmdLine);
	
	if( !net )
	{
		LogError("segnet:  failed to initialize segNet\n");
		return 0;
	}

	// set alpha blending value for classes that don't explicitly already have an alpha	
	net->SetOverlayAlpha(cmdLine.GetFloat("alpha", 150.0f));

	// get the desired overlay/mask filtering mode
	const segNet::FilterMode filterMode = segNet::FilterModeFromStr(cmdLine.GetString("filter-mode", "linear"));

	// get the visualization flags
	const uint32_t visualizationFlags = segNet::VisualizationFlagsFromStr(cmdLine.GetString("visualize", DEFAULT_VISUALIZATION));

	// get the object class to ignore (if any)
	const char* ignoreClass = cmdLine.GetString("ignore-class", "void");

	
	
	/*
	 * processing loop
	 */
	while( !signal_recieved )
	{
		// capture next image image
		pixelType* imgInput = NULL;

		if( !input->Capture(&imgInput, 1000) )
		{
			// check for EOS
			if( !input->IsStreaming() )
				break; 

			LogError("segnet:  failed to capture video frame\n");
			continue;
		}

		// allocate buffers for this size frame
		if( !allocBuffers(input->GetWidth(), input->GetHeight(), visualizationFlags) )
		{
			LogError("segnet:  failed to allocate buffers\n");
			continue;
		}

		// process the segmentation network
		if( !net->Process(imgInput, input->GetWidth(), input->GetHeight(), ignoreClass) )
		{
			LogError("segnet:  failed to process segmentation\n");
			continue;
		}
		
		// generate overlay
		if( visualizationFlags & segNet::VISUALIZE_OVERLAY )
		{
			if( !net->Overlay(imgOverlay, overlaySize.x, overlaySize.y, filterMode) )
			{
				LogError("segnet:  failed to process segmentation overlay.\n");
				continue;
			}
		}

		// generate mask
		if( visualizationFlags & segNet::VISUALIZE_MASK )
		{
			if( !net->Mask(imgMask, maskSize.x, maskSize.y, filterMode) )
			{
				LogError("segnet:-console:  failed to process segmentation mask.\n");
				continue;
			}
		}

		// generate composite
		if( (visualizationFlags & segNet::VISUALIZE_OVERLAY) && (visualizationFlags & segNet::VISUALIZE_MASK) )
		{
			CUDA(cudaOverlay(imgOverlay, overlaySize, imgComposite, compositeSize, 0, 0));
			CUDA(cudaOverlay(imgMask, maskSize, imgComposite, compositeSize, overlaySize.x, 0));
		}

		// render outputs
		if( output != NULL )
		{
			output->Render(imgOutput, outputSize.x, outputSize.y);

			// update the status bar
			char str[256];
			sprintf(str, "TensorRT %i.%i.%i | %s | Network %.0f FPS", NV_TENSORRT_MAJOR, NV_TENSORRT_MINOR, NV_TENSORRT_PATCH, net->GetNetworkName(), net->GetNetworkFPS());
			output->SetStatus(str);

			// check if the user quit
			if( !output->IsStreaming() )
				signal_recieved = true;
		}

		// wait for the GPU to finish		
		CUDA(cudaDeviceSynchronize());

		// print out timing info
		net->PrintProfilerTimes();
	}
	

	/*
	 * destroy resources
	 */
	LogVerbose("segnet:  shutting down...\n");
	
	SAFE_DELETE(input);
	SAFE_DELETE(output);
	SAFE_DELETE(net);

	CUDA_FREE_HOST(imgMask);
	CUDA_FREE_HOST(imgOverlay);
	CUDA_FREE_HOST(imgComposite);

	LogVerbose("segnet:  shutdown complete.\n");
	return 0;
}

```

Editar el CMakeList.txt con lo siguiente

```
# require CMake 2.8 or greater
cmake_minimum_required(VERSION 2.8)

# declare my-segmentation project
project(my-segmentation)

# import jetson-inference and jetson-utils packages.
# note that if you didn't do "sudo make install"
# while building jetson-inference, this will error.
find_package(jetson-utils)
find_package(jetson-inference)

# CUDA and Qt4 are required
find_package(CUDA)
find_package(Qt4)

# setup Qt4 for build
include(${QT_USE_FILE})
add_definitions(${QT_DEFINITIONS})

# compile the my-segmentation program
cuda_add_executable(my-segmentation my-segmentation.cpp)

# link my-segmentation to jetson-inference library
target_link_libraries(my-segmentation jetson-inference)
```

Compilar el código con los siguientes comandos

```
$ cmake .
$ make
```

Ejecutar el programa con el siguiente comando

```
$ ./my-segmentation /dev/video0
```

En este caso abrirá la webcam, se pueden introducir las mismas variables que con el programa precompilado